In [24]:
#import libraries
import xgboost as xg
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
import numpy as np
from sklearn.metrics import mean_squared_error
import matplotlib as plt
from scipy.stats import norm
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedStratifiedKFold, KFold
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten, BatchNormalization
from sklearn.metrics import r2_score
import data_ML as ML
import pickle

np.set_printoptions(suppress=True)

In [25]:
df = ML.getdata(False)


In [33]:
def modifydata(df):
    data = []
    df_calls = df[df.optionType == 1].drop(columns='optionType')
    df_american_calls = df_calls[df_calls.method == 1].drop(columns='method')
    # df_american_calls = df_american_calls.drop(columns='index')
    df_american_calls = df_american_calls[df_american_calls.lastPrice > 5]
    # df_american_calls = df_american_calls.drop(columns='index')
    y = np.asarray(df_american_calls[['lastPrice']])
    df_american_calls = df_american_calls.drop(columns='lastPrice')

    for line in range(len(df_american_calls)):
        data_tmp = df_american_calls.iloc[line]
        data.append(data_tmp)
    data = np.asarray(data)

    return data, y

X, y = modifydata(df)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=3)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1743, 5) (582, 5) (1743, 1) (582, 1)


In [34]:

# df_train, y_train, full_data2 = modifydata(df_train)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=3)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1743, 5) (582, 5) (1743, 1) (582, 1)


In [28]:
# Define the model
model_american_calls = Sequential()
model_american_calls.add(Dense(512, input_dim=5, activation='relu'))
model_american_calls.add(Dropout(0.2))
model_american_calls.add(Dense(256, activation='relu'))
model_american_calls.add(Dropout(0.2))
model_american_calls.add(Dense(256, activation='relu'))
model_american_calls.add(Dropout(0.2))
model_american_calls.add(Dense(64, activation='relu'))
model_american_calls.add(Dense(1))
model_american_calls.compile(optimizer='adam', loss='mse')

history = model_american_calls.fit(X_train, y_train, epochs=100, validation_split=0.2, batch_size=128)

# model_american_calls.save("model_american_calls.h5")
# model_american_calls.save("model_american_puts.h5")



Epoch 1/100
11/11 [==============================] - 0s 7ms/step - loss: 76808.2031 - val_loss: 75314.9531
Epoch 2/100
11/11 [==============================] - 0s 4ms/step - loss: 64722.8359 - val_loss: 72780.4141
Epoch 3/100
11/11 [==============================] - 0s 4ms/step - loss: 61903.2773 - val_loss: 72385.1016
Epoch 4/100
11/11 [==============================] - 0s 4ms/step - loss: 61861.3789 - val_loss: 71001.5156
Epoch 5/100
11/11 [==============================] - 0s 4ms/step - loss: 63189.4414 - val_loss: 70270.2812
Epoch 6/100
11/11 [==============================] - 0s 4ms/step - loss: 61659.1523 - val_loss: 70783.3203
Epoch 7/100
11/11 [==============================] - 0s 4ms/step - loss: 60490.9688 - val_loss: 69713.7656
Epoch 8/100
11/11 [==============================] - 0s 3ms/step - loss: 60020.3398 - val_loss: 70596.4453
Epoch 9/100
11/11 [==============================] - 0s 3ms/step - loss: 60187.3047 - val_loss: 69500.4844
Epoch 10/100
11/11 [=================

In [29]:
#prediction
import pandas as pd
y_pred = model_american_calls.predict(X_test)


db = pd.DataFrame({'prediction':y_pred.tolist(),'real':y_test.tolist()})
error = (y_pred - y_test)/y_test
db = db.assign(error = error)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(rmse)
db.to_csv('./Data/MLPdata.csv')
db




19/19 [==============================] - 0s 672us/step
97.09296815585658


,prediction,real,error
0,[98.74156188964844],[111.7],-0.116011
1,[21.857589721679688],[22.6],-0.032850
2,[94.75263977050781],[102.57],-0.076215
3,[108.00997924804688],[191.77],-0.436773
4,[86.59449005126953],[368.8],-0.765199
...,...,...,...
577,[107.82339477539062],[122.32],-0.118514
578,[182.186279296875],[199.67],-0.087563
579,[190.9197998046875],[197.47],-0.033171
580,[44.18874740600586],[360.0],-0.877253


In [30]:
y_test.shape

(582, 1)

In [31]:
DTR1 = DecisionTreeRegressor()
# cvs_DTR = cross_val_score(DTR, df_train, y_train, scoring='neg_root_mean_squared_error', cv=5)
# DTR_score = cvs_DTR.mean()
# print(DTR_score)

DTR1.fit(X_train, y_train)
DTR1_pred = DTR1.predict(X_test)
DTR1_rmse = np.sqrt(mean_squared_error(y_test, DTR1_pred))
DTR1_rmse

36.22851763506706

In [32]:
y_pred = pd.DataFrame(DTR1_pred, columns=['prediction'])
final_test = pd.DataFrame(y_test, columns=['real'])

final_df = pd.concat((final_test, y_pred), axis=1)
final_df

,real,prediction
0,111.70,113.30
1,22.60,22.90
2,102.57,95.50
3,191.77,211.94
4,368.80,366.58
...,...,...
577,122.32,135.61
578,199.67,201.30
579,197.47,206.33
580,360.00,354.00


In [23]:

XGBr = xg.XGBRegressor(learning_rate=0.1, n_estimators=900)
cvs_XGBr = cross_val_score(XGBr, X_train, y_train, scoring='neg_root_mean_squared_error', cv=KFold(n_splits=10))
XGBr_score = cvs_XGBr.mean()
print(XGBr_score)


-28.400139537023126


In [12]:
XGBr.fit(X_train, y_train)
XGBr_pred = XGBr.predict(X_test)
XGBr_rmse = np.sqrt(mean_squared_error(y_test, XGBr_pred))
XGBr_rmse
saved_model = pickle.dumps(XGBr)

In [13]:
XGBr_saved_model = pickle.loads(saved_model)
def xgb_prediction(K, T, S_0, sigma , r):
    data = np.array([[K, T, S_0, sigma , r]])
    prediction = XGBr_saved_model.predict(data)
    return prediction



In [14]:
print(xgb_prediction(230, 246/365, 189.71, 0.3318, 0.269))


[24.331013]
